# Implementing elementary cellular automaton (Rule 90) in Q♯

*Dmytro Fedoriaka, December 2021*

This notebook supplements my blog post "Implementing cellular automaton in Q#". It contains Q# code implementing "[Rule 90](https://en.wikipedia.org/wiki/Rule_90)" cellular automaton in Q#, and some simulation experiments with it.

It's part of my project on quantum circuits for ECAs, more details [here](https://github.com/fedimser/quant_comp/tree/master/cellular_automata).


You can download and execute this Jupyter Notebook. For that, in addition to Python and Jupyter, you need to install Q# kernel following [these instructions](https://docs.microsoft.com/en-us/azure/quantum/install-command-line-qdk#jupyter-notebooks-using-the-net-cli). 

## 1. The Q# code

In the code below:

* Operation "Rule90" applies Rule 90 to given register of qubits.
* Operation "MeasureAndPrint" measures register (in computational basis) and prints result.
* Operation "EvolveOneBit" build initial state where there is only one qubit in state $| 1 \rangle$, and all others in state $| 0 \rangle$, evolves it for given number of steps, measures and print the result.
* Operation "EvolveSuperposition" build superposition of two initial states with exactly one qubit in state $| 1 \rangle$ (both with weights $\frac{1}{\sqrt{2}}$, evolves it for given number of states, measures and prints the result. 


In [1]:
open Microsoft.Quantum.Diagnostics;

operation LeftShift(qs: Qubit[]) : Unit {
    let N = Length(qs);
    for i in 0..N-2 {
        SWAP(qs[i], qs[i+1]);
    }
}

operation Rule90(qs: Qubit[]) : Unit {
    let N = Length(qs);
    for i in 3..N {
        CNOT(qs[N-i], qs[N-i+2]);
    }
    for i in 1..(N/2)-1 {
        CNOT(qs[2*i], qs[0]);
    }
    LeftShift(qs);    
}

operation MeasureAndPrint(qs: Qubit[]) : Unit {
    let N = Length(qs);
    mutable result = new Int[N];
    for i in 0..N-1 {
        if(M(qs[i]) == One) {
            set result w/= i <- 1;
        }
    }
    Message($"{result}");
}

operation EvolveOneBit(N: Int, pos: Int, steps: Int) : Unit {
    use qs = Qubit[N];
    X(qs[pos]);
    MeasureAndPrint(qs);
    for i in 1..steps {
        Rule90(qs);
        MeasureAndPrint(qs);
    }
    ResetAll(qs);
}

operation EvolveSuperposition(N: Int, pos1: Int, pos2: Int, steps: Int, attempts: Int) : Unit {
    use qs = Qubit[10];
        for attempt in 1..attempts {
        H(qs[pos1]);
        CNOT(qs[pos1], qs[pos2]);
        X(qs[pos2]);
        for i in 1..steps {
            Rule90(qs);
        }
        MeasureAndPrint(qs);
        ResetAll(qs);
    }
}

LeftShift, Rule90, MeasureAndPrint, EvolveOneBit, EvolveSuperposition

## 2. Simulating single-state cell

Let's simulate initial states with only 1 qubit set to $| 1 \rangle$:

In [2]:
%simulate EvolveOneBit N=10 pos=2 steps=10

[0,0,1,0,0,0,0,0,0,0]
[0,1,0,1,0,0,0,0,0,0]
[1,0,0,0,1,0,0,0,0,0]
[0,1,0,1,0,1,0,0,0,0]
[1,0,0,0,0,0,1,0,0,0]
[0,1,0,0,0,1,0,1,0,0]
[1,0,1,0,1,0,0,0,1,0]
[0,0,0,0,0,1,0,1,0,1]
[0,0,0,0,1,0,0,0,0,0]
[0,0,0,1,0,1,0,0,0,0]
[0,0,1,0,0,0,1,0,0,0]


()

In [3]:
%simulate EvolveOneBit N=16 pos=8 steps=10

[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
[0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0]
[0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0]
[0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0]
[0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0]
[0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0]
[0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0]
[0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
[1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]


()

As expected, we are getting the "Sierpinski triangle" pattern.

## 3. Simulating superposition of 2 single-state cells

Now, let's simulate evolution of superposition. Here, we start with initial state
$\frac{1}{\sqrt{2}} |0000100000 \rangle + \frac{1}{\sqrt{2}} |0000010000 \rangle$.

In theory, we expect the result to be superposition of two basis states, each of which represents evolution of superposed initial states.

To confirm that, we simulate evolution of the same superposition 10 times, and each time measure the result.

In [4]:
%simulate EvolveSuperposition N=10 pos1=4 pos2=5 steps=5 attempts=10

[1,0,1,0,0,0,0,0,1,0]
[0,1,0,0,0,0,0,1,0,1]
[1,0,1,0,0,0,0,0,1,0]
[0,1,0,0,0,0,0,1,0,1]
[0,1,0,0,0,0,0,1,0,1]
[0,1,0,0,0,0,0,1,0,1]
[0,1,0,0,0,0,0,1,0,1]
[1,0,1,0,0,0,0,0,1,0]
[0,1,0,0,0,0,0,1,0,1]
[1,0,1,0,0,0,0,0,1,0]


()

As expected, we see only 2 different outcomes, and they appear with equal frequency. And we can check that those outcomes are exactly results of evolutions of states 0000100000 and 0000010000.